In [ ]:
## For colab - you must install the GPU version of this each time
!pip install faiss-gpu

In [ ]:
import faiss
import os
import json
import numpy as np
import torch
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms
from torchvision import datasets, models
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings('ignore')
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from google.cloud import storage
#from torch.utils.tensorboard import SummaryWriter
# %load_ext tensorboard
import datetime
import time
import torch.nn as nn

from torchvision.models.resnet import *
from torchvision.models.resnet import BasicBlock, Bottleneck

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('drive/MyDrive/Colab Notebooks/AICORE/FAISS')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
## Leverage the process_img function

def process_img(image):
    pil_to_tensor = transforms.ToTensor()
    resize = transforms.Resize((225,225))
    img = Image.open(image).convert('RGB')

    features = pil_to_tensor(img)
    features = resize(features)
    features = torch.unsqueeze(features, dim=0)
    #print(features.shape)
    return features

In [ ]:
## Read the embeddings JSON file and convert to a dictionary

with open('final_embeddings.json', "r") as json_file:
    data_dict = json.load(json_file)

index = faiss.IndexFlatL2(2048)   # build the index, d=size of vectors
# here we assume xb contains a n-by-d numpy matrix of type float32

## Create a flattened array of float32 vectors
embeddings_array = np.array(list(data_dict.values()), dtype='float32')
## Create a maching array of the vector ids (based on the filenames)
embeddings_ids = np.array(list(data_dict.keys()))
## Create the FAISS index by using the add function
index.add(embeddings_array)
print(index.is_trained)

True


In [ ]:
## Created a classifier based on the RESNET50 pretrained model

class ItemFeatureExtractor(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
        #self.resnet50 = model
        self.resnet50.fc = torch.nn.Linear(2048,13)

    def forward(self, X):
        return F.softmax(self.resnet50(X))

In [ ]:
## Instantiate the model class as the feature extractor with the original parameters
model = ItemFeatureExtractor()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

## Load the final training weights
checkpoint = torch.load('final_weights/weights.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [ ]:
## Remove the final classification layer and make a 2048 embedding output available
model = nn.Sequential(*list(model.resnet50.children())[:-1])

In [ ]:
## Test FAISS by creating a set of query embeddings using the model output and process_image function
query_embeddings = model(process_img('pytorch_images_tv/train/appliances/16.jpg'))

In [ ]:
## Flatten the output
query_embeddings = query_embeddings.view(query_embeddings.size(0), -1)

In [ ]:
## Convert the output to an float 32 array
query_vector = np.array(list(query_embeddings.tolist()), dtype='float32')

In [ ]:
query_vector.shape

(1, 2048)

In [ ]:

## Test the index search
D, I = index.search(query_vector.reshape(1, -1), 4)

In [ ]:
I

array([[814, 942, 759, 843]])

In [ ]:
embeddings_ids[759]

'pytorch_images_tv/train/appliances/11-Best-Small-Kitchen-Appliances-Banner-MJ'